In [2]:
import pandas as pd
import sklearn

In [5]:
#loading intermediate dataset ukbms and haduk, created in  HADUK_intermediate.py and UKBMS_HADUK_Exploration.ipynb
df = pd.read_csv('BioD_year_site_latlong.csv')
df.drop(df.columns[0], inplace = True)
df

KeyError: "['Unnamed: 0'] not found in axis"